---
# BaselHack 2025 Pax Challenge Group A2 - Neural Network
---



### 1: Setup and Data Preprocessing
This cell includes code to set up the environment, including mounting Google Drive and loading the dataset from a specified path.
It preprocesses the data by scaling numerical features and encoding categorical features using StandardScaler and OneHotEncoder, respectively.
The dataset is split into features (X) and target (y), followed by a further split into training and testing sets (X_train, X_test, y_train, y_test).
The cell also creates a DataFrame from the processed features for display purposes, which helps in verifying the preprocessing steps.

In [7]:
# Cell 1: Setup and Preprocessing with all features categorical

from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

drive.mount('/content/drive', force_remount=True)

# Path to your file
file_path = '/content/drive/My Drive/Hackathon Project Pax/synthetic_underwriting_offering1.csv'
df = pd.read_csv(file_path)

# --- Categorize BMI ---
df['bmi_category'] = pd.cut(
    df['bmi'],
    bins=[0, 18.5, 25, 30, 100],
    labels=['underweight', 'normal', 'overweight', 'obese'],
    right=True, include_lowest=True
)

# --- Categorize AGE ---
df['age_category'] = pd.cut(
    df['age'],
    bins=[0, 24, 34, 44, 54, 64, 200],
    labels=['young', 'early_adult', 'mid_adult', 'mature', 'senior', 'elderly'],
    right=True, include_lowest=True
)

# --- Categorize doctor visits (right-skewed) ---
df['doctor_visits_category'] = pd.cut(
    df['doctor_visits_per year'],
    bins=[-0.1, 0, 2, 5, 10, 1000],
    labels=['none', 'low', 'moderate', 'high', 'very_high'],
    right=True, include_lowest=True
)

# --- Define final feature set (all categorical now) ---
feature_columns = [
    'offering_id',              # treated as categorical ID
    'smoking',
    'age_category',
    'bmi_category',
    'extreme_sport',
    'doctor_visits_category',
    'occupational_risk_level'
]

categorical_cols = feature_columns.copy()

# Convert offering_id to string so OneHotEncoder treats it as categorical
df['offering_id'] = df['offering_id'].astype(str)

# --- Preprocessor ---
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# --- Prepare data ---
X = df[feature_columns]
y = df['risk_score'].values  # still regression target

X_processed = preprocessor.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=23
)

print("✅ Data transformed — all features categorical!")
print("Shape of X_processed:", X_processed.shape)


Mounted at /content/drive
✅ Data transformed — all features categorical!
Shape of X_processed: (895, 23)


### 2: Neural Network Model Training and Saving

This cell defines the architecture of the neural network model using Keras, including adding Dense layers and specifying activation functions.
The model is compiled and trained on the training dataset (X_train, y_train).
After training, the model is saved to Google Drive for future use.
The model's performance is evaluated on the test dataset (X_test, y_test), and the test accuracy is printed, providing insight into the model's predictive capabilities.



In [8]:
# Cell 2: Neural Network Model Training, Saving, and Test Evaluation (Regression)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Convert X_train and X_test to dense arrays if necessary
if hasattr(X_train, "toarray"):
    X_train = X_train.toarray()
if hasattr(X_test, "toarray"):
    X_test = X_test.toarray()

# Define model (for regression)
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Single neuron output for regression

# Compile model with regression-appropriate loss
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Check data shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

# Train the model
history = model.fit(X_train, y_train, epochs=89, batch_size=10, validation_split=0.2, verbose=1)
print("✅ Model successfully trained on the training set!")

# Save the trained model
model.save('/content/drive/My Drive/Hackathon Project Pax/pax_underwriting_regression_model.keras')
print("💾 Model saved successfully!")

# Evaluate model on the test set
loss, mae = model.evaluate(X_test, y_test)
print(f"📊 Test Mean Absolute Error (MAE): {mae:.3f}")
print(f"📉 Test Mean Squared Error (MSE): {loss:.3f}")


X_train shape: (716, 23)
y_train shape: (716,)
Epoch 1/89


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 245231.5312 - mae: 462.9165 - val_loss: 254969.5000 - val_mae: 476.5838
Epoch 2/89
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 237522.5000 - mae: 456.4596 - val_loss: 211947.1094 - val_mae: 429.8078
Epoch 3/89
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 182820.6250 - mae: 385.7536 - val_loss: 107143.7578 - val_mae: 290.2091
Epoch 4/89
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 80087.7656 - mae: 240.8237 - val_loss: 33722.6680 - val_mae: 148.5220
Epoch 5/89
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 27079.4883 - mae: 131.9242 - val_loss: 23488.3008 - val_mae: 123.3682
Epoch 6/89
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 25476.4141 - mae: 128.7283 - val_loss: 20155.1289 - val_mae: 114.2046
Epoch 7/89
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 20794.4004 - mae: 117.4083 - val_loss: 17638.8047 - val_mae: 107.2296
Epoch 8/89
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 19899.2910 - mae: 112.7980 - val_loss: 15494.3857 -

### Next Step: Inspecting Weights

In [10]:
# Cell 3: Show learned weights per feature (all categorical setup)

import numpy as np
import pandas as pd

# --- 1️⃣ Get first-layer weights and biases ---
weights, biases = model.layers[0].get_weights()
print("Weight matrix shape:", weights.shape)   # (n_features, n_neurons)
print("Bias vector shape:", biases.shape)

# --- 2️⃣ Retrieve one-hot feature names from the preprocessor ---
cat_features = list(
    preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
)

# --- 3️⃣ Compute average absolute weight per input feature ---
# Each input connects to multiple neurons; average absolute value gives an importance estimate
mean_abs_weights = np.mean(np.abs(weights), axis=1)

feature_importance = pd.DataFrame({
    'Feature': cat_features,
    'Average_Abs_Weight': mean_abs_weights
}).sort_values(by='Average_Abs_Weight', ascending=False)

# --- 4️⃣ Display top weights ---
print("\n🔎 Feature Importance based on first-layer weights:")
print(feature_importance.to_string(index=False))

# --- 5️⃣ Save results to Drive ---
output_path = '/content/drive/My Drive/Hackathon Project Pax/feature_importance.csv'
feature_importance.to_csv(output_path, index=False)
print(f"\n📁 Feature importance exported to Drive: {output_path}")


Weight matrix shape: (23, 64)
Bias vector shape: (64,)

🔎 Feature Importance based on first-layer weights:
                         Feature  Average_Abs_Weight
doctor_visits_category_very_high            0.687765
               extreme_sport_yes            0.516965
    occupational_risk_level_high            0.515362
              bmi_category_obese            0.446159
                     smoking_yes            0.376022
            age_category_elderly            0.341437
             age_category_senior            0.328015
         bmi_category_overweight            0.302139
                   offering_id_1            0.275884
     doctor_visits_category_high            0.262040
  occupational_risk_level_medium            0.231905
             age_category_mature            0.230998
                      smoking_no            0.197491
          age_category_mid_adult            0.194885
     occupational_risk_level_low            0.183876
             bmi_category_normal            0

### 3: DMN Decision Function

The cell contains the DMN decision function, which implements the business logic to determine the risk category ('Not Reject', or 'Reject') based on inputs such as age, country, and disease.
This function is essential for the initial screening of the input data based on predefined rules and criteria.

In [ ]:
# Cell 3: Updated DMN Decision Function

def dmn_decision(age_group):
    if age_group == '200-215':
        return 'No'




### 4: User Input and DMN Processing
In this cell, user input data is defined in a structured format.
The input data is then processed through the DMN decision function defined in Cell 3.
Depending on the output of the DMN function, this cell prepares the data for further processing by the neural network or concludes with a decision.

In [ ]:
# Cell 4: Input User Data and Process through DMN

user_input = {
    'age_group': '60-75',
    'education_group': 'higher education',
    'job': 'retired',
    'marital': 'married',
    'default': 'no',
    'previous_group': '4 or more',
    'poutcome': 'success',
    'has_loan': 'yes',
    'Economics_state': -0.8
}

try:
    dmn_output = dmn_decision(user_input['age_group'])

except ValueError as e:
    print(e)
    dmn_output = None

if dmn_output == 'No':
    final_decision = 'No'
else:
    user_input['y'] = dmn_output
    #If we want to delete a column which is made redundant after the DMN (before entering the NN):
    #del user_input['nationality']  # This removes the 'nationality' key for example
    print("Processed through DMN:", user_input)


Processed through DMN: {'age_group': '60-75', 'education_group': 'higher education', 'job': 'retired', 'marital': 'married', 'default': 'no', 'previous_group': '4 or more', 'poutcome': 'success', 'has_loan': 'yes', 'Economics_state': -0.8, 'y': None}


### 5: Preprocessing User Input for Neural Network
This cell includes a function to preprocess the user input data (after DMN processing) to make it suitable for prediction by the neural network.
The preprocessing steps typically involve scaling and encoding similar to what was done for the training data, ensuring consistency in data format.

In [ ]:
# Cell 5: Preprocess User Input for Neural Network

import pandas as pd

def preprocess_input(input_data, preprocessor):
    input_df = pd.DataFrame([input_data])
    return preprocessor.transform(input_df)

# Assuming dmn_output is defined from previous steps
if dmn_output != 'No':
    processed_input = preprocess_input(user_input, preprocessor)
else:
    processed_input = None

# If we're debugging, print to check the processed input:
if processed_input is not None:
    print("Processed input shape:", processed_input.shape)


Processed input shape: (1, 39)


### 6: Neural Network Prediction
This cell uses the pre-trained neural network model to predict the final decision based on the preprocessed user input data.
The prediction is interpreted by mapping the neural network's output to the respective class names.
The final decision, based on the neural network's output, is printed. This decision can be either the predicted class directly or a derived decision such as "Reject" based on the DMN output.

In [ ]:
# Cell 6: Revised Neural Network Prediction

import numpy as np

if processed_input is not None and dmn_output != 'No':
    # Predict using the model and find the index of the predicted class
    nn_prediction = model.predict(processed_input)
    predicted_class_index = np.argmax(nn_prediction, axis=1)

    # Since `predicted_class_index` is a 1-element array, extract the first value
    predicted_class = unique_classes[predicted_class_index[0]]

    final_decision = predicted_class
else:
    final_decision = "No"

print("Neural Network Determined Class:", final_decision)




1/1 [==============================] - 0s 22ms/step
Neural Network Determined Class: yes


### Cell 7: Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Make predictions using the model on the test set
y_pred = model.predict(X_test)
# Convert predicted probabilities to predicted class indices
y_pred_indices = np.argmax(y_pred, axis=1)
# Convert the actual one-hot encoded labels to class indices for comparison
y_test_indices = np.argmax(y_test, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(y_test_indices, y_pred_indices)

# Extract TN, FP, FN, and TP from the confusion matrix
# This code assumes a binary classification problem
if cm.shape == (2, 2):
    tn, fp, fn, tp = cm.ravel()
else:
    raise ValueError("Confusion matrix shape is not binary. Make sure your problem is binary classification.")

# Calculate the metrics
N = tn + fp + fn + tp
E = (fp + fn) / N  # Error rate
R = (tp + tn) / N  # Recognition rate
precision = tp / (tp + fp) if (tp + fp) != 0 else 0
recall = tp / (tp + fn) if (tp + fn) != 0 else 0
f1_metric = 2 / ((1/precision)+(1/recall))

# Print out results
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")
print(f"True Negatives (TN): {tn}")

print(f"\nError rate (E): {E:.3f}")
print(f"Recognition rate (R): {R:.3f}")
print(f"Precision (p): {precision:.3f}")
print(f"Recall (r): {recall:.3f}")
print(f"F1 Metric (r): {f1_metric:.3f}")

print("\n")
print("\n")
print("\n---OPTIONAL---")
# Optionally, print a full classification report for more detailed evaluation
print("\nClassification Report:")
print(classification_report(y_test_indices, y_pred_indices, target_names=unique_classes))


258/258 [==============================] - 1s 2ms/step
False Positives (FP): 157
False Negatives (FN): 720
True Positives (TP): 167
True Negatives (TN): 7194

Error rate (E): 0.106
Recognition rate (R): 0.894
Precision (p): 0.515
Recall (r): 0.188
F1 Metric (r): 0.276





---OPTIONAL---

Classification Report:
              precision    recall  f1-score   support

          no       0.91      0.98      0.94      7351
         yes       0.52      0.19      0.28       887

    accuracy                           0.89      8238
   macro avg       0.71      0.58      0.61      8238
weighted avg       0.87      0.89      0.87      8238

